# Appraisal Annotation Extraction from Inception Files
* Based on extract_information_from_JSON_appraisal function from https://github.com/linatal/UNSC_multilayer_corpus/blob/main/json2csv_merge.py

## Requirements

### Installs

In [ ]:
!pip install alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.6 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=57305bc0c6874aeb829e839a91be8d8674d0ba0fbea3369fadd8e5c38563b50f
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


### General Requirements

In [ ]:
from alive_progress import alive_bar
import json
import pandas as pd
from pathlib import Path
import re
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Paths Extraction

In [ ]:
# Creating DF for storage of extracted annotations
columns = ['filename_appraisal',
               'fileid',
               'sentence',
               'sentence token',
               'sentence character offset begin',
               'sentence character offset end',
               'annotated text',
               'attitude',
               'polarity',
               'appraisal character offset',
               'attitude target',
               'id annotated attitude',
               'relations_target_attitude']
df = pd.DataFrame(columns=columns)

In [ ]:
# Setting directories
print('retrieving data: appraisal \n')
appraisals = "appraisal"
appraisals_directory = "XX" # ADD PATH
output_file_appraisal = "/content/dataset/output_appraisals.csv"

retrieving data: appraisal 



In [ ]:
# Function for Path Retrieval
# Based on * Based on extract_information_from_JSON_appraisal function from https://github.com/linatal/UNSC_multilayer_corpus/blob/main/json2csv_merge.py
def get_paths(directory, task):
    """
    Args:
        directory: directory to curation
        task: task type ("appraisal", "conflict", "arguments")

    Returns: list with all paths to CURATION_USER.json or admin.json files
    """

    _temp_file_list = []
    current_directory = Path.cwd()
    task_directory = current_directory / Path(directory)

    folders = [f for f in task_directory.iterdir() if f.is_dir()]
    #files = [f for f in task_directory.iterdir() if f.is_file()]

    for folder in folders:
        # Match name with the regex pattern
        pattern = r"(UNSC_\d+_SPV\.\d+_spch\d+)"
        match = re.search(pattern, str(folder))  # Check for a match
        if match:  # Only proceed if the match exists
            name = match.group(1)  # Extract the matched name
            # Construct paths based on the task type
            if task in ["appraisal", "conflict"]:
                filename_appr = 'admin.json'
                file_path_appr = task_directory / Path(folder) / Path(filename_appr)
                _temp_file_list.append((name, file_path_appr))
            elif task == "arguments":
                filename_arg = 'admin.json'
                file_path_arg = task_directory / Path(folder) / Path(filename_arg)
                _temp_file_list.append((name, file_path_arg))
        else:
            print(f"No match found for folder: {folder}")  # Debugging information

    return _temp_file_list


In [ ]:
input_files_appraisal = get_paths(appraisals_directory, appraisals)

In [ ]:
input_files_appraisal

[('UNSC_2014_SPV.7219_spch004',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/appraisal-annotation-in-unsc-speeches/annotation/UNSC_2014_SPV.7219_spch004_sentsplit_United_Kingdom_Of_Great_Britain_And_Northern_Ireland.json/admin.json')),
 ('UNSC_2014_SPV.7154_spch004',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/appraisal-annotation-in-unsc-speeches/annotation/UNSC_2014_SPV.7154_spch004_sentsplit_Russian_Federation.json/admin.json')),
 ('UNSC_2014_SPV.7138_spch005',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/appraisal-annotation-in-unsc-speeches/annotation/UNSC_2014_SPV.7138_spch005_sentsplit_France.json/admin.json')),
 ('UNSC_2016_SPV.7658_spch026',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/appraisal-annotation-in-uns

## Function for Appraisal Annotations Extractions, includes:
* Attitude labels, polarity labels,attitude annotated text, attitude target and Id of relation between attitude fragment and attitude target.

In [ ]:
# Funtion for extracting appraisal annotations
# Based on extract_information_from_JSON_appraisal function from https://github.com/linatal/UNSC_multilayer_corpus/blob/main/json2csv_merge.py
def extract_information_from_JSON_appraisal(files, output_file):
    """
    Extracts information from JSON files related to appraisal annotation.

    Args:
        files: List of paths in the appraisal corpus.
        output_file: Name of output file.

    Returns:
        DataFrame with extracted annotation data.
    """

    columns = ['filename_appraisal',
               'fileid',
               'sentence',
               'sentence token',
               'sentence character offset begin',
               'sentence character offset end',
               'annotated text',
               'attitude',
               'polarity',
               'appraisal character offset',
               'attitude target',
               'id annotated attitude',
               'target relation ID',
               'ID attitude',
               'ID attitude target',
               'relations_comp']  # Added column for the ID of annotated attitude


    df = pd.DataFrame(columns=columns)
    error_list = []

    with alive_bar(len(files), force_tty=True) as bar:
        for document in files:
            try:
                with open(document[1], 'r') as file:
                    data = json.load(file)
            except Exception as e:
                print(f"Error reading {document[1]}: {e}")
                continue

            _temp_filename = ""
            _temp_sentences = []
            _temp_tokens = []
            _temp_text = ''
            _temp_annotations = []
            _temp_targets = []

            # ---------- NEW  ----------
            _temp_relations = []  # Stores attitude-target relations
            # ------------------------------

            for item in data['%FEATURE_STRUCTURES']:
                if item["%TYPE"] == "de.tudarmstadt.ukp.dkpro.core.api.metadata.type.DocumentMetaData":
                    _temp_filename = item.get("documentTitle", "Unknown")
                elif item['%TYPE'] == 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence':
                    _temp_sentences.append((item['begin'], item['end']))
                elif item['%TYPE'] == 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token':
                    _temp_tokens.append((item['begin'], item['end']))
                elif item['%TYPE'] == 'uima.cas.Sofa':
                    _temp_text = item.get('sofaString', '')
                elif item['%TYPE'] == 'webanno.custom.AppraisalTheory':
                    begin = item['begin']
                    end = item['end']
                    attitude = item.get('Attitude', None)
                    polarity = item.get('Polarity', None)
                    target = item.get('Attitude_target', None)
                    _temp_id = item.get('%ID', None)  # Extract the ID for the annotated attitude\


                    #if attitude is not None or polarity is not None:
                    _temp_annotations.append((begin, end, attitude, polarity, _temp_id))  # Store the ID with annotations

                    _temp_targets.append((begin, end, target))
                    print(type(_temp_relations))

                # ------------NEW RELATIONS ------------
                elif  item['%TYPE'] == 'webanno.custom.AttitudeTargetRelations' and item.get("target_relation") == "explicit_target":
                      relation_id = item.get('%ID', None)
                      governor_id = item.get('@Governor', None)  # ID of Attitude fragment
                      dependent_id = item.get('@Dependent', None)  # ID of Attitude target

                      _temp_relations.append((relation_id, governor_id, dependent_id))

                 # ------------------------------------



            #_temp_relations = {}
            _temp_argument_targets = {}
            _temp_new_ID = {}


            # Loop over the feature structures to find attitude relations
            for item in data.get('%FEATURE_STRUCTURES', []):
              if item.get('%TYPE') == 'webanno.custom.AttitudeTargetRelations':  # Looking for Attitude relations type
                try:
                    # Extracting information for Attitude relations
                    relation_id = item.get('%ID')
                    target_relation = item.get('target_relation', 'no target')  # e.g., 'explicit_target'
                    governor_id = item.get('@Governor')
                    dependent_id = item.get('@Dependent')

                    # Store the relation in _temp_relations
                    '''if _temp_relations.get(_temp_new_ID.get(governor_id)) is None:
                        _temp_relations[_temp_new_ID.get(governor_id)] = [
                            (_temp_new_ID.get(dependent_id), relation_id, target_relation)
                        ]
                    else:
                        _temp_relations[_temp_new_ID.get(governor_id)].append(
                            (_temp_new_ID.get(dependent_id), relation_id, target_relation)
                        )'''

                    # Store the target relations
                    if _temp_argument_targets.get(_temp_new_ID.get(governor_id)) is None:
                        _temp_argument_targets[_temp_new_ID.get(governor_id)] = [target_relation]
                    else:
                        _temp_argument_targets[_temp_new_ID.get(governor_id)].append(target_relation)

                except KeyError as e:
                    print(f"Error processing item {item}: Missing key {e}")
                    continue

            counter_token = 0
            counter_annotation = 0
            counter_target = 0  # Added for targets

            for sent in _temp_sentences:
                sent_begin, sent_end = sent
                _temp_token_sent_list = []
                _temp_annotation_text_list = []
                _temp_annotation_attitude_list = []
                _temp_annotation_polarity_list = []
                _temp_annotation_offset_list = []
                _temp_target_list = []
                _temp_annotation_id_list = []  # List to store the IDs of annotated attitudes


                while _temp_tokens[counter_token][1] <= sent[1] and len(_temp_tokens) - 1 != counter_token:
                  _temp_token_sent_list.append(_temp_text[_temp_tokens[counter_token][0]:_temp_tokens[counter_token][1]])
                  counter_token += 1

                # Annotation Extraction
                while counter_annotation < len(_temp_annotations) and _temp_annotations[counter_annotation][1] <= sent_end:
                    begin, end, attitude, polarity, ann_id = _temp_annotations[counter_annotation]
                    annotated_text = _temp_text[begin:end]
                    _temp_annotation_text_list.append(annotated_text)
                    _temp_annotation_attitude_list.append(attitude)
                    _temp_annotation_polarity_list.append(polarity)
                    _temp_annotation_offset_list.append((begin, end))
                    _temp_annotation_id_list.append(ann_id)  # Add the annotated attitude ID
                    counter_annotation += 1

                # Target Extraction
                while counter_target < len(_temp_targets) and _temp_targets[counter_target][1] <= sent_end:
                    begin, end, target = _temp_targets[counter_target]
                    target_text = _temp_text[begin:end] if target else "None"
                    _temp_target_list.append(target_text)
                    print("_temp_target_list: ", _temp_target_list)
                    counter_target += 1


                # NEW
                # Convert _temp_relations into a dictionary for fast lookup
                relation_dict = {gov_id: (rel_id, dep_id) for rel_id, gov_id, dep_id in _temp_relations}
                # Ensure all lists are aligned


                # Ensure all lists are aligned
                for i in range(len(_temp_annotation_text_list)):
                    attitude_id = _temp_annotation_id_list[i]  # Get the ID of the annotated attitude

                    # Get relation info if attitude_id matches a Governor in relations
                    if attitude_id in relation_dict:
                        target_relation_id, attitude_target_id = relation_dict[attitude_id]
                    else:
                        target_relation_id, attitude_target_id = None, None  # No relation found

                    # Check if "attitude target" is not None
                    if _temp_target_list[i] != "None" and _temp_target_list[i] is not None:
                        # If attitude target exists, set other columns to None
                        annotated_text = None
                        attitude = None
                        polarity = None
                        id_annotated_attitude = None
                    else:
                        # If no attitude target exists, keep the original values
                        annotated_text = _temp_annotation_text_list[i]
                        attitude = _temp_annotation_attitude_list[i]
                        polarity = _temp_annotation_polarity_list[i]
                        id_annotated_attitude = attitude_id

                        #print("_temp_target_list[i]: ", _temp_target_list[i])

                    # Add the row to the DataFrame
                    df = pd.concat([df, pd.DataFrame([{
                        'filename_appraisal': _temp_filename,
                        'fileid': document[0],
                        'sentence': _temp_text[sent_begin:sent_end],
                        'sentence token': str(_temp_token_sent_list),
                        'sentence character offset begin': sent_begin,
                        'sentence character offset end': sent_end,
                        'annotated text': annotated_text,
                        'attitude': attitude,
                        'polarity': polarity,
                        'appraisal character offset': _temp_annotation_offset_list[i],
                        'attitude target': _temp_target_list[i] if i < len(_temp_target_list) else "None",
                        'id annotated attitude': id_annotated_attitude,  # Add the ID of the annotated attitude
                        'target relation ID': target_relation_id,  # Add the relation ID
                        'ID attitude': attitude_id,  # The attitude itself
                        'ID attitude target': attitude_target_id,  # The ID of the attitude target
                        'relations_comp': [target_relation_id,
                                          [attitude_id, annotated_text],
                                          [attitude_target_id, _temp_target_list[i]]]  # Corrected relations_comp extraction
                    }])], ignore_index=True)
            bar()

    df.to_csv(output_file, index=False)
    print("Data extraction complete.")
    return df




In [ ]:
# Calling function
df_funct_2 = extract_information_from_JSON_appraisal(input_files_appraisal, output_file_appraisal)

on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: <class 'list'>
on 0: _temp_target_list:  ['None']
on 0: _temp_target_list:  ['None']
on 0: _temp_target_list:  ['None', 'None']
on 0: _temp_target_list:  ['None']
on 0: _temp_target_list:  ['None']
on 0: _temp_target_list:  ['None', 'None']
on 0: _temp_target_list:  ['None', 'None', 'None']
on 0: _temp_target_list:  ['None', 'None', 'None', 'None']
on 0: _temp_target_list:  ['None', 'None', 'None', 'None', 'Mr. Feltman for his update']
on 0: _temp_target_list:  ['None', 'None', 'None', 'None', 'Mr. Feltman for his update', 'situation']
on 0: _temp

# WORKING FUNCTION
  * Falta solo ajustar la sacada del offset del attitude target

In [ ]:
def extract_information_from_JSON_appraisal_target(files, output_file):
    """
    Extracts information from JSON files related to appraisal annotation.

    Args:
        files: List of paths in the appraisal corpus.
        output_file: Name of output file.

    Returns:
        DataFrame with extracted annotation data.
    """

    columns = ['filename_appraisal',
               'fileid',
               'sentence',
               'sentence token',
               'sentence character offset begin',
               'sentence character offset end',
               'annotated text',
               'attitude',
               'polarity',
               'appraisal character offset',
               'attitude target',
               'id annotated attitude',
               'target relation ID',
               'ID attitude',
               'ID attitude target',
               'relations_comp']  # Added column for the ID of annotated attitude


    df = pd.DataFrame(columns=columns)
    error_list = []

    # Dictionary to store the Attitude Target text by its ID
    attitude_target_text = {}

    with alive_bar(len(files), force_tty=True) as bar:
        for document in files:
            try:
                with open(document[1], 'r') as file:
                    data = json.load(file)
            except Exception as e:
                print(f"Error reading {document[1]}: {e}")
                continue

            _temp_filename = ""
            _temp_sentences = []
            _temp_tokens = []
            _temp_text = ''
            _temp_annotations = []
            _temp_targets = []

            _temp_relations = []  # Stores attitude-target relations

            for item in data['%FEATURE_STRUCTURES']:
                if item["%TYPE"] == "de.tudarmstadt.ukp.dkpro.core.api.metadata.type.DocumentMetaData":
                    _temp_filename = item.get("documentTitle", "Unknown")
                elif item['%TYPE'] == 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence':
                    _temp_sentences.append((item['begin'], item['end']))
                elif item['%TYPE'] == 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token':
                    _temp_tokens.append((item['begin'], item['end']))
                elif item['%TYPE'] == 'uima.cas.Sofa':
                    _temp_text = item.get('sofaString', '')
                elif item['%TYPE'] == 'webanno.custom.AppraisalTheory':
                    begin = item['begin']
                    end = item['end']
                    attitude = item.get('Attitude', None)
                    polarity = item.get('Polarity', None)
                    target = item.get('Attitude_target', None)
                    _temp_id = item.get('%ID', None)  # Extract the ID for the annotated attitude

                    _temp_annotations.append((begin, end, attitude, polarity, _temp_id))  # Store the ID with annotations
                    _temp_targets.append((begin, end, target))

                # AttitudeTargetRelations: Store the Dependent value (Attitude_target) in a dictionary
                elif item['%TYPE'] == 'webanno.custom.AttitudeTargetRelations' and item.get("target_relation") == "explicit_target":
                    relation_id = item.get('%ID', None)
                    governor_id = item.get('@Governor', None)
                    dependent_id = item.get('@Dependent', None)  # This is the ID for the attitude target

                    # Store the attitude target text
                    attitude_target_text[dependent_id] = _temp_text[item['begin']:item['end']]

                    _temp_relations.append((relation_id, governor_id, dependent_id))

            # Process sentences and annotations
            counter_token = 0
            counter_annotation = 0
            counter_target = 0

            for sent in _temp_sentences:
                sent_begin, sent_end = sent
                _temp_token_sent_list = []
                _temp_annotation_text_list = []
                _temp_annotation_attitude_list = []
                _temp_annotation_polarity_list = []
                _temp_annotation_offset_list = []
                _temp_target_list = []
                _temp_annotation_id_list = []

                while _temp_tokens[counter_token][1] <= sent[1] and len(_temp_tokens) - 1 != counter_token:
                    _temp_token_sent_list.append(_temp_text[_temp_tokens[counter_token][0]:_temp_tokens[counter_token][1]])
                    counter_token += 1

                while counter_annotation < len(_temp_annotations) and _temp_annotations[counter_annotation][1] <= sent_end:
                    begin, end, attitude, polarity, ann_id = _temp_annotations[counter_annotation]
                    annotated_text = _temp_text[begin:end]
                    _temp_annotation_text_list.append(annotated_text)
                    _temp_annotation_attitude_list.append(attitude)
                    _temp_annotation_polarity_list.append(polarity)
                    _temp_annotation_offset_list.append((begin, end))
                    _temp_annotation_id_list.append(ann_id)
                    counter_annotation += 1

                while counter_target < len(_temp_targets) and _temp_targets[counter_target][1] <= sent_end:
                    begin, end, target = _temp_targets[counter_target]
                    target_text = _temp_text[begin:end] if target else "None"
                    _temp_target_list.append(target_text)
                    counter_target += 1

                relation_dict = {gov_id: (rel_id, dep_id) for rel_id, gov_id, dep_id in _temp_relations}

                for i in range(len(_temp_annotation_text_list)):
                    attitude_id = _temp_annotation_id_list[i]

                    if attitude_id in relation_dict:
                        target_relation_id, attitude_target_id = relation_dict[attitude_id]
                    else:
                        target_relation_id, attitude_target_id = None, None

                    # If attitude_target_id is not None, get the associated attitude target text from the dictionary
                    if attitude_target_id is not None and attitude_target_id in attitude_target_text:
                        attitude_target_text_value = attitude_target_text[attitude_target_id]
                    else:
                        attitude_target_text_value = "None"

                    # Prepare the DataFrame row
                    df = pd.concat([df, pd.DataFrame([{
                        'filename_appraisal': _temp_filename,
                        'fileid': document[0],
                        'sentence': _temp_text[sent_begin:sent_end],
                        'sentence token': str(_temp_token_sent_list),
                        'sentence character offset begin': sent_begin,
                        'sentence character offset end': sent_end,
                        'annotated text': _temp_annotation_text_list[i],
                        'attitude': _temp_annotation_attitude_list[i],
                        'polarity': _temp_annotation_polarity_list[i],
                        'appraisal character offset': _temp_annotation_offset_list[i],
                        'attitude target': attitude_target_text_value,  # Add the attitude target text
                        'id annotated attitude': attitude_id,
                        'target relation ID': target_relation_id,
                        'ID attitude': attitude_id,
                        'ID attitude target': attitude_target_id,
                        'relations_comp': [target_relation_id,
                                           [attitude_id, _temp_annotation_text_list[i]],
                                           [attitude_target_id, attitude_target_text_value]]
                    }])], ignore_index=True)

                bar()

    df.to_csv(output_file, index=False)
    print("Data extraction complete.")
    return df


In [ ]:
df_funct_3 = extract_information_from_JSON_appraisal_target(input_files_appraisal, output_file_appraisal)

|████████████████████████████████████████✗︎ (!) 2442/87 [2807%] in 30.2s (80.72/s
Data extraction complete.


In [ ]:
df_funct_3

,filename_appraisal,fileid,sentence,sentence token,sentence character offset begin,sentence character offset end,annotated text,attitude,polarity,appraisal character offset,attitude target,id annotated attitude,target relation ID,ID attitude,ID attitude target,relations_comp
0,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,UNSC_2014_SPV.7219_spch004,We meet today in the most tragic of circumstan...,"['We', 'meet', 'today', 'in', 'the', 'most', '...",0,50,most tragic of circumstances,appreciation,negative,"(21, 49)",circumstances,928,987,928,955,"[987, [928, most tragic of circumstances], [95..."
1,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,UNSC_2014_SPV.7219_spch004,"Our deepest, most heartfelt condolences go to ...","['Our', 'deepest', ',', 'most', 'heartfelt', '...",189,325,"Our deepest, most heartfelt condolences go to",affect,negative,"(189, 234)",the families of the victims and to the people ...,929,988,929,956,"[988, [929, Our deepest, most heartfelt condol..."
2,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,UNSC_2014_SPV.7219_spch004,"In the United Kingdom, we are mourning the los...","['In', 'the', 'United', 'Kingdom', ',', 'we', ...",326,442,we are mourning the loss,affect,negative,"(349, 373)",of at least nine British citizens who were pas...,930,989,930,957,"[989, [930, we are mourning the loss], [957, o..."
3,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,UNSC_2014_SPV.7219_spch004,We welcome the indications from the Ukrainian ...,"['We', 'welcome', 'the', 'indications', 'from'...",1147,1270,We welcome,affect,positive,"(1147, 1157)",the indications from the Ukrainian authorities...,931,996,931,962,"[996, [931, We welcome], [962, the indications..."
4,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,UNSC_2014_SPV.7219_spch004,We welcome the indications from the Ukrainian ...,"['We', 'welcome', 'the', 'indications', 'from'...",1147,1270,should have a central role to play in ensuring...,judgement,positive,"(1033, 1145)",The International Civil Aviation Organization,932,995,932,961,"[995, [932, should have a central role to play..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,UNSC_2014_SPV.7154_spch002_sentsplit_UN.txt,UNSC_2014_SPV.7154_spch002,"Russia, which shares a long border area with U...","['Russia', ',', 'which', 'shares', 'a', 'long'...",3345,3498,crisis,appreciation,negative,"(2501, 2507)",crisis,710,744,710,702,"[744, [710, crisis], [702, crisis]]"
2831,UNSC_2014_SPV.7154_spch002_sentsplit_UN.txt,UNSC_2014_SPV.7154_spch002,"Russia, which shares a long border area with U...","['Russia', ',', 'which', 'shares', 'a', 'long'...",3345,3498,that the Ukraine crisis will continue to deepe...,None,None,"(3010, 3147)",None,711,None,711,None,"[None, [711, that the Ukraine crisis will cont..."
2832,UNSC_2014_SPV.7154_spch002_sentsplit_UN.txt,UNSC_2014_SPV.7154_spch002,"Russia, which shares a long border area with U...","['Russia', ',', 'which', 'shares', 'a', 'long'...",3345,3498,teeters on the very brink.,judgement,negative,"(3172, 3198)",Ukraine,712,743,712,713,"[743, [712, teeters on the very brink.], [713,..."
2833,UNSC_2014_SPV.7154_spch002_sentsplit_UN.txt,UNSC_2014_SPV.7154_spch002,"Russia, which shares a long border area with U...","['Russia', ',', 'which', 'shares', 'a', 'long'...",3345,3498,Ukraine,None,None,"(3164, 3171)",None,713,None,713,None,"[None, [713, Ukraine], [None, None]]"


In [ ]:
df_funct_3.to_excel("Appraisal Annotations with targets.xlsx")

## Version de extraccion completa de las relaciones

In [ ]:
def extract_information_attitude_target_relations_2(files, output_file):
    columns = ['fileid', 'filename', 'speaker', 'speechnumber', 'attitude_targets_relations']
    df = pd.DataFrame(columns=columns)
    error_list = []

    with alive_bar(len(files), force_tty=True) as bar:
        for document in files:
            fileid = document[0]  # Extract file ID
            filename = Path(document[1]).parts[9]  # Extract filename without extension

            # Extract speaker from filename
            match = re.search(r'_([^_]+)\.json$', filename)
            speaker = match.group(1) if match else "Unknown"

            # Extract speech number using regex
            match = re.search(r'_spch(\d{3})', fileid)
            speech_number = match.group(1) if match else "Unknown"

            with open(document[1], 'r') as file:
                data = json.load(file)

            _temp_text = ''
            _temp_attitude_annotations = {}
            _temp_attitude_targets = {}
            _temp_relations = {}

            for item in data['%FEATURE_STRUCTURES']:
                if item['%TYPE'] == 'uima.cas.Sofa':
                    _temp_text = item['sofaString']
                elif item['%TYPE'] == 'webanno.custom.AppraisalTheory':
                    if 'Attitude' in item and 'Polarity' in item:
                        _temp_attitude_annotations[item['%ID']] = (
                            item['begin'], item['end'], item['Attitude'], item['Polarity']
                        )
                    elif 'Attitude_target' in item:
                        _temp_attitude_targets[item['%ID']] = (
                            item['begin'], item['end'], 'Attitude_target'
                        )
                elif item['%TYPE'] == 'webanno.custom.AttitudeTargetRelations':
                    try:
                        governor_id = item['@Governor']
                        dependent_id = item['@Dependent']
                        relation_label = item.get('target_relation', 'no relation')
                        _temp_relations[governor_id] = (dependent_id, relation_label)
                    except KeyError:
                        error_list.append((document[0], item['begin'], item['end']))

            attitude_targets_relations = []

            for att_id, (att_start, att_end, att_label, att_polarity) in _temp_attitude_annotations.items():
                if att_id in _temp_relations:
                    dependent_id, relation_label = _temp_relations[att_id]
                    if dependent_id in _temp_attitude_targets:
                        tgt_start, tgt_end, tgt_label = _temp_attitude_targets[dependent_id]
                        attitude_targets_relations.append([
                            [_temp_text[att_start:att_end], f'{att_label}+{att_polarity}'],
                            [_temp_text[tgt_start:tgt_end], tgt_label],
                            relation_label
                        ])

            new_row = pd.DataFrame([[document[0], filename, speaker, speech_number, attitude_targets_relations]], columns=columns)
            df = pd.concat([df, new_row], ignore_index=True)
            bar()

        df.to_csv(output_file, sep='\t', index=False)
        print(f"DataFrame saved to {output_file}")
        print('Error List:', error_list)

    return df

In [ ]:
df_relations = extract_information_attitude_target_relations_2(input_files_appraisal, output_file_appraisal)

on 87: DataFrame saved to /content/dataset/output_appraisals.csv
on 87: Error List: []
|████████████████████████████████████████| 87/87 [100%] in 0.7s (127.96/s)      


In [ ]:
df_relations

,fileid,filename,speaker,speechnumber,attitude_targets_relations
0,UNSC_2014_SPV.7219_spch004,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,Ireland,004,"[[[most tragic of circumstances, appreciation+..."
1,UNSC_2014_SPV.7154_spch004,UNSC_2014_SPV.7154_spch004_sentsplit_Russian_F...,Federation,004,"[[[emergency meeting, appreciation+negative], ..."
2,UNSC_2014_SPV.7138_spch005,UNSC_2014_SPV.7138_spch005_sentsplit_France.json,France,005,[]
3,UNSC_2016_SPV.7658_spch026,UNSC_2016_SPV.7658_spch026_sentsplit_Egypt.json,Egypt,026,[]
4,UNSC_2014_SPV.7154_spch021,UNSC_2014_SPV.7154_spch021_sentsplit_Russian_F...,Federation,021,"[[[I would like to, affect+positive], [share w..."
...,...,...,...,...,...
82,UNSC_2014_SPV.7219_spch025,UNSC_2014_SPV.7219_spch025_sentsplit_Indonesia...,Indonesia,025,"[[[I thank, affect+positive], [the President f..."
83,UNSC_2016_SPV.7658_spch013,UNSC_2016_SPV.7658_spch013_sentsplit_United_St...,America,013,[]
84,UNSC_2016_SPV.7643_spch014,UNSC_2016_SPV.7643_spch014_sentsplit_Spain.json,Spain,014,[]
85,UNSC_2014_SPV.7165_spch007,UNSC_2014_SPV.7165_spch007_sentsplit_Argentina...,Argentina,007,"[[[would like to thank, affect+positive], [Mr...."


In [ ]:
# Function to extract elements and expand rows
def expand_attitude_relations(df):
    new_rows = []

    for _, row in df.iterrows():
        fileid, filename, speaker, speechnumber, relations = row

        for rel in relations:
            if len(rel) >= 3 and isinstance(rel[0], list) and isinstance(rel[1], list):
                attitude_fragment, attitude_label = rel[0]
                attitude_target, _ = rel[1]
                target_type = rel[2]

                new_rows.append({
                    "fileid": fileid,
                    "filename": filename,
                    "speaker": speaker,
                    "speechnumber": speechnumber,
                    "attitude_fragment": attitude_fragment,
                    "attitude_label": attitude_label,
                    "attitude_target": attitude_target,
                    "target_type": target_type
                })

    return pd.DataFrame(new_rows)

# Transforming the dataframe
expanded_df = expand_attitude_relations(df_relations)


                          fileid  \
0     UNSC_2014_SPV.7219_spch004   
1     UNSC_2014_SPV.7219_spch004   
2     UNSC_2014_SPV.7219_spch004   
3     UNSC_2014_SPV.7219_spch004   
4     UNSC_2014_SPV.7219_spch004   
...                          ...   
1119  UNSC_2014_SPV.7154_spch002   
1120  UNSC_2014_SPV.7154_spch002   
1121  UNSC_2014_SPV.7154_spch002   
1122  UNSC_2014_SPV.7154_spch002   
1123  UNSC_2014_SPV.7154_spch002   

                                               filename  speaker speechnumber  \
0     UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...  Ireland          004   
1     UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...  Ireland          004   
2     UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...  Ireland          004   
3     UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...  Ireland          004   
4     UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...  Ireland          004   
...                                                 ...      ...          ...   


In [ ]:
expanded_df.to_excel("Appraisal_annotations_target_fragment_label.xlsx")